# Importy:

In [1]:
# import os
# import json
# import math
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()
# sns.set()


## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
# import torchvision
# from torchvision.utils import save_image
# from torchvision.datasets import FashionMNIST
# from torchvision.transforms import v2
# from torchvision.datasets import ImageFolder
# from torch.nn.functional import one_hot

# from pytorch_fid import fid_score

# from collections import Counter
from sklearn.model_selection import train_test_split

/tmp/ipykernel_6115/294544994.py:13: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export


In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cuda


In [3]:
import transformers
from transformers import *
from datasets import load_dataset
import evaluate
from tqdm import tqdm

/home/kuba/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kuba/.local/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


# Wczytywanie danych

In [4]:
PATH = '../../data'
reviews = pd.read_csv(f"{PATH}/train_data.csv")
reviews.head()

,review,rating
0,location not palace excellent hotel booke dthe...,4
1,respite definitely not place stay looking ultr...,3
2,stunning truly memorable spot right beach nusa...,4
3,solid business hotel near embassy stayed hotel...,3
4,nice place make sure lock money warning money ...,3


In [5]:
reviews['rating'].value_counts()

rating
4    7243
3    4831
2    1747
1    1434
0    1137
Name: count, dtype: int64

Z komórki wyżej wynika, że dane treningowe są niezbilansowane. \
Znamy 2 główne metody "naprawy" tego problemu: undersampling i oversampling. \
Zdecydowaliśmy się zastosować metodę undersamplingu, w nadziei, że prawie 1200 wierszy dla każdej z klas wystarczy.

In [6]:
X = reviews.filter(['review'])
y = reviews.filter(['rating'])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [8]:
from imblearn.under_sampling import RandomUnderSampler

In [9]:
sampler = RandomUnderSampler(sampling_strategy="not minority")
resampled_train_data, resampled_train_classes = sampler.fit_resample(X_train, y_train)
resampled_test_data, resampled_test_classes = sampler.fit_resample(X_test, y_test)

In [10]:
resampled_train_classes.value_counts()

rating
0         790
1         790
2         790
3         790
4         790
Name: count, dtype: int64

In [11]:
resampled_test_classes.value_counts()

rating
0         347
1         347
2         347
3         347
4         347
Name: count, dtype: int64

TODO

Łącznie, resampled_train_classes oraz resampled_test_classes mają po 1137 w każdej z klas. \
\
Najpierw podzieliliśmy początkowy zbiór testowy na nowe dane testowe i walidacyjne, a następnie zbilansowaliśmy dane testowe otrzymane w wyniku podziału. Kolejność tych operacji jest ważna, ponieważ chcemy, aby zbiór walidacyjny nie zawierał żadnych zmian (aby zbiór walidacyjny był jak najbliższy "rzeczywistym" danym).

In [12]:
resampled = pd.merge(resampled_train_data, resampled_train_classes, left_index=True, right_index=True)
resampled_test = pd.merge(resampled_test_data, resampled_test_classes, left_index=True, right_index=True)

In [13]:
resampled.head()

,review,rating
4635,disappointed service there-the restaurant open...,0
13177,"run thought picking cheap hotel save lot, not,...",0
5778,"rating suspect, hotel sofitel june 6-7 power w...",0
3526,"words, ok stay riu hotel punta cana, twice sta...",0
982,n't fooled price location grade room damrak ho...,0


In [14]:
resampled['rating'].value_counts()

rating
0    790
1    790
2    790
3    790
4    790
Name: count, dtype: int64

# Modele

## Model Rekurencyjny

TODO

In [15]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
corpus = api.load('text8')
gensim_model = Word2Vec(corpus)

In [16]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
class LSTMRegressor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size, emb_weights, bidirectional = False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.embeddings = nn.Embedding.from_pretrained(emb_weights)
        self.embeddings.requires_grad = False
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional=bidirectional, batch_first=False)
        self.fc = nn.Linear(hidden_size*self.bidirectional, out_size)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, len_x, hidden):
        x = self.embeddings(x)
        x = torch.transpose(x,0,1)
        all_outputs, hidden = self.lstm(x, hidden)
        all_outputs = torch.transpose(all_outputs,0,1)
        last_seq_items = all_outputs[range(all_outputs.shape[0]), len_x]
        out = last_seq_items
        x = self.fc(out)
        return x, hidden

In [17]:
emb_weights = torch.FloatTensor(gensim_model.wv.vectors)
embedding = nn.Embedding.from_pretrained(emb_weights)
tokenizer = gensim_model.wv.key_to_index

In [18]:
lstm_model = LSTMRegressor(100, 100, 1, 5, emb_weights).to(device)
lstm_model

LSTMRegressor(
  (embeddings): Embedding(71290, 100)
  (lstm): LSTM(100, 100)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)

## Bag of Words

In [19]:
from bs4 import BeautifulSoup
import re

Aby usprawnić działanie metody BoW, zastosujemy tzw. "stopwords".

In [20]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/kuba/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

"Data cleaning"

In [22]:
def review_to_words(raw_review):
    """Function to convert a review to a string of words.
    The input is a single string (a raw moviw review), and the output is a single string (a preprocessed movie review)"""
    review_text = BeautifulSoup(raw_review, 'lxml').get_text()
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    words = letters_only.lower().split()
    stops = set(stopwords.words('english'))
    meaningful_words = [word for word in words if not word in stops]
    return " ".join(meaningful_words)

In [23]:
i = 0
clean_reviews = []
num_reviews = reviews['review'].size

for review in reviews['review']:
    i += 1
    if i % 200 == 0:
        print('Review {} of {}'.format(i, num_reviews))
    clean_reviews.append(review_to_words(review))

Review 200 of 16392
Review 400 of 16392
Review 600 of 16392
Review 800 of 16392
Review 1000 of 16392


/tmp/ipykernel_6115/3593606712.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(raw_review, 'lxml').get_text()


Review 1200 of 16392
Review 1400 of 16392
Review 1600 of 16392
Review 1800 of 16392
Review 2000 of 16392
Review 2200 of 16392
Review 2400 of 16392
Review 2600 of 16392
Review 2800 of 16392
Review 3000 of 16392
Review 3200 of 16392
Review 3400 of 16392
Review 3600 of 16392
Review 3800 of 16392
Review 4000 of 16392
Review 4200 of 16392
Review 4400 of 16392
Review 4600 of 16392
Review 4800 of 16392
Review 5000 of 16392
Review 5200 of 16392
Review 5400 of 16392
Review 5600 of 16392
Review 5800 of 16392
Review 6000 of 16392
Review 6200 of 16392
Review 6400 of 16392
Review 6600 of 16392
Review 6800 of 16392
Review 7000 of 16392
Review 7200 of 16392
Review 7400 of 16392
Review 7600 of 16392
Review 7800 of 16392
Review 8000 of 16392
Review 8200 of 16392
Review 8400 of 16392
Review 8600 of 16392
Review 8800 of 16392
Review 9000 of 16392
Review 9200 of 16392
Review 9400 of 16392
Review 9600 of 16392
Review 9800 of 16392
Review 10000 of 16392
Review 10200 of 16392
Review 10400 of 16392
Review 106

In [24]:
print('Creating the bag of words...')
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's bag of words tool.
vectorizer = CountVectorizer(analyzer = 'word',
                            tokenizer = None,
                            preprocessor = None,
                            stop_words = None,
                            max_features = 1000)
# fit_transform() does two functions: First, it fits the model
# and learns the vocaulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of strings.
train_data_features = vectorizer.fit_transform(clean_reviews)

# Numpy arrays are easy to work with, so convert the result to an array
train_data_features = train_data_features.toarray()
print('Bag of words completed')

Creating the bag of words...
Bag of words completed


In [25]:
vocab = vectorizer.get_feature_names_out()
print(len(vocab))

1000


In [26]:
flatten_train_classes = resampled_train_classes.to_numpy().flatten()
flatten_test_classes = resampled_test_classes.to_numpy().flatten()

In [27]:
train_indices = np.random.rand(len(reviews))>0.3
train_data = torch.from_numpy(train_data_features).float()[train_indices]
train_targets = torch.from_numpy(reviews['rating'].values[train_indices]).long()

test_data = torch.from_numpy(train_data_features[~train_indices]).float()
test_targets = torch.from_numpy(reviews['rating'].values[~train_indices]).long()

In [28]:
# train_data = torch.from_numpy(train_data_features).float()
# train_targets = torch.from_numpy(flatten_train_classes).long()

# test_data = 

In [29]:
train_dataset = torch.utils.data.TensorDataset(train_data, train_targets)
test_dataset = torch.utils.data.TensorDataset(test_data, test_targets)

In [30]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

In [31]:
class BoWClassifier(nn.Module):
    def __init__(self):
        super(BoWClassifier, self).__init__()
        self.lin1 =nn.Linear(1000, 800)
        self.act1 =nn.LeakyReLU()
        self.lin2 =nn.Linear(800, 500)
        self.act2 =nn.LeakyReLU()
        self.lin3 = nn.Linear(500, 100)
        self.act3 = nn.LeakyReLU()
        # self.lin4 = nn.Linear(200, 50)
        # self.act4 = nn.LeakyReLU()
        self.lin4 =nn.Linear(100, 5)


    def forward(self, x):
        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)
        x = self.lin3(x)
        x = self.act3(x)
        x = self.lin4(x)
        # x = self.act4(x)
        # x = self.lin5(x)
        return x
bow_model = BoWClassifier().to(device)
bow_model

BoWClassifier(
  (lin1): Linear(in_features=1000, out_features=800, bias=True)
  (act1): LeakyReLU(negative_slope=0.01)
  (lin2): Linear(in_features=800, out_features=500, bias=True)
  (act2): LeakyReLU(negative_slope=0.01)
  (lin3): Linear(in_features=500, out_features=100, bias=True)
  (act3): LeakyReLU(negative_slope=0.01)
  (lin4): Linear(in_features=100, out_features=5, bias=True)
)

In [32]:
def get_accuracy(model, data_loader):
    correct = 0
    total = 0
    model.eval() #*********#
    for imgs, labels in data_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        output = model(imgs)
        pred = output.max(1, keepdim=True)[1] # get the index of the max logit
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(bow_model.parameters(), lr=0.001)

iters = []
losses = []
train_acc = []
val_acc = []
for n in range(100):
    epoch_losses = []
    for x, labels in iter(train_loader):
        x, labels = x.to(device), labels.to(device)
        bow_model.train()
        out = bow_model(x).squeeze()

        loss = criterion(out, labels)
        loss.backward()
        epoch_losses.append(loss.item())
        optimizer.step()
        optimizer.zero_grad()

    loss_mean = np.array(epoch_losses).mean()
    iters.append(n)
    losses.append(loss_mean)
    test_acc = get_accuracy(bow_model, test_loader)
    print(f"Epoch {n} loss {loss_mean:.3} test_acc: {test_acc:.3}")
    train_acc.append(get_accuracy(bow_model, train_loader)) # compute training accuracy
    # val_acc.append(test_acc)  # compute validation accuracy


print("Final Training Accuracy: {}".format(train_acc[-1]))
# print("Final Validation Accuracy: {}".format(val_acc[-1]))

Epoch 0 loss 1.0 test_acc: 0.612
Epoch 1 loss 0.775 test_acc: 0.61
Epoch 2 loss 0.589 test_acc: 0.59
Epoch 3 loss 0.328 test_acc: 0.566
Epoch 4 loss 0.139 test_acc: 0.59
Epoch 5 loss 0.0694 test_acc: 0.569
Epoch 6 loss 0.052 test_acc: 0.584
Epoch 7 loss 0.0466 test_acc: 0.568
Epoch 8 loss 0.032 test_acc: 0.579
Epoch 9 loss 0.0356 test_acc: 0.58
Epoch 10 loss 0.0333 test_acc: 0.58
Epoch 11 loss 0.0275 test_acc: 0.563
Epoch 12 loss 0.0245 test_acc: 0.561
Epoch 13 loss 0.015 test_acc: 0.565
Epoch 14 loss 0.0371 test_acc: 0.566
Epoch 15 loss 0.02 test_acc: 0.566
Epoch 16 loss 0.0114 test_acc: 0.564
Epoch 17 loss 0.00985 test_acc: 0.56
Epoch 18 loss 0.017 test_acc: 0.552
Epoch 19 loss 0.0331 test_acc: 0.56
Epoch 20 loss 0.0235 test_acc: 0.561
Epoch 21 loss 0.0125 test_acc: 0.554
Epoch 22 loss 0.0139 test_acc: 0.55
Epoch 23 loss 0.00712 test_acc: 0.56
Epoch 24 loss 0.00333 test_acc: 0.555
Epoch 25 loss 0.0101 test_acc: 0.563
Epoch 26 loss 0.00849 test_acc: 0.557
Epoch 27 loss 0.0157 test_acc

## Gotowe modele z biblioteki sklearn

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import FunctionTransformer

In [35]:
pipelineMultinomialNB = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('MNB', MultinomialNB())
])

pipelineComplementNB = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('CNB', ComplementNB())
])

pipelineLinearSVM = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('LSVC', LinearSVC())
])

pipelineSVM = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('SVC', SVC())
])

In [36]:
# print(type(resampled_train_data))
# print(type(resampled_train_classes))

In [37]:
# print(type(resampled_train_data))
# print(type(resampled_train_classes))
# print(resampled_train_data.shape)
# print(resampled_train_classes.shape)
# print(resampled_train_data.values.reshape(-1))
# print(resampled_train_classes.values.reshape(-1))
# print(resampled_train_data.values.shape)
# print(resampled_train_classes.values.reshape(-1,1).shape)

In [38]:
pipelineMultinomialNB.fit(resampled_train_data.values.flatten(), resampled_train_classes.values.flatten())
predictMNB = pipelineMultinomialNB.predict(X_test.to_numpy().flatten())
print(accuracy_score(y_test, predictMNB))

0.5079300528670191


In [39]:
pipelineComplementNB.fit(resampled_train_data.values.flatten(), resampled_train_classes.values.flatten())
predictCNB = pipelineComplementNB.predict(X_test.to_numpy().flatten())
print(accuracy_score(y_test, predictCNB))

0.5453436356242375


In [40]:
pipelineLinearSVM.fit(resampled_train_data.values.flatten(), resampled_train_classes.values.flatten())
predictLSVM = pipelineLinearSVM.predict(X_test.to_numpy().flatten())
print(accuracy_score(y_test, predictLSVM))

0.5461569743798292


In [41]:
pipelineSVM.fit(resampled_train_data.values.flatten(), resampled_train_classes.values.flatten())
predictSVM = pipelineSVM.predict(X_test.to_numpy().flatten())
print(accuracy_score(y_test, predictSVM))

0.5673037820252135


Sposród przetestowanych przez nas gotowych modeli dostarczonych w bibliotece scikit-learn, najlepsze wyniki daje model SVM. Zauważyliśmy przy tym, że przeprowadzenie całości ostatniego pipeline'u zajmuje znacznie więcej czasu (~25-30 sek.) w porównaniu do pozostałych trzech modeli (~0.7 sek.).